# Data Cleaning

![alt text](./images/USA_zones.png)
<center>(Figure 1) Impact Zones</center>

**Storm Status Codes**
* WV - Tropical Wave
* TD - Tropical Depression
* TS - Tropical Storm
* HU - Hurricane
* EX - Extratropical cyclone
* SD - Subtropical depression (winds <34 kt)
* SS - Subtropical storm (winds >34 kt)
* LO - A low pressure system not fitting any of above descriptions
* DB - non-tropical Disturbance not have a closed circulation

"Storm Event Chain" refers to the measurements of a particular storm

Import modules

In [1]:
import pandas as pd
import numpy as np

Import data

In [2]:
with open('./data/hurdat2.csv', 'r') as f:
    file_data = [row.strip().split(',') for row in f]
df = pd.DataFrame(file_data)
df.drop([2] + list(range(7,21)), axis='columns', inplace=True)
cols = ['date', 'time', 'status', 'lat', 'lon', 'max_wind']
data = pd.DataFrame(np.matrix(df), columns=cols)

Data cleaning functions

In [3]:
def lon_convert(lon):
    lon = lon.strip()
    lon_sign = 1
    if lon[-1] == 'W':
        lon_sign = -1
    return lon_sign * float(lon[:-1])

In [4]:
def time_filter(t):
    try:
        int(t)
        return(t)
    except:
        return('9999')

In [5]:
def time_convert(time_str):
    try:
        return(int(time_str.strip()))
    except:
        pass

Data cleaning

In [6]:
data.date = data.date.apply(lambda x: '99999999' if x[0] == 'A' else x.strip())
data.time = data.time.apply(lambda x: x.strip())
data.status = data.status.apply(lambda x: 'NE' if x == '' else x.strip())
data.lat = data.lat.apply(lambda x: '00.0N' if x is None else x.strip())
data.lat = data.lat.apply(lambda x: float(x.strip()[:-1]))
data.lon = data.lon.apply(lambda x: '00.0W' if x is None else x.strip())
data.lon = data.lon.apply(lambda x: lon_convert(x))
data = data[data.lon > -200]    # Dropping noise
data.max_wind = data.max_wind.apply(lambda x: 0 if x is None else int(x.strip()))
data.time = data.time.apply(lambda x: time_filter(x))
data.loc[:, 'year'] = data.date.apply(lambda x: int(x[0:4]))
data.loc[:, 'month'] = data.date.apply(lambda x: int(x[4:6]))
data.loc[:, 'day'] = data.date.apply(lambda x: int(x[6:8]))
data.drop('date', axis='columns', inplace=True)
data.loc[:, 'hour'] = data.time.apply(lambda x: int(x[0:2]))
data.loc[:, 'minute'] = data.time.apply(lambda x: int(x[2:4]))
data.drop(['time', 'minute'], axis='columns', inplace=True)
data.max_wind = data.max_wind.apply(lambda x: 30 if x == -99 else x)
data.loc[:, 'td'] = data.hour.diff()    # Time delta
data.td = data.td.apply(lambda x: x + 24 if x < 0 else x)
data.td = data.td.apply(lambda x: 6 if x < 0 else x)
data.reset_index(inplace=True, drop=True)

Import Hurricane Harvey and Irma data

In [7]:
HU_2017 = pd.read_csv('./data/hurricanes_2017.csv')
HU_2017.lon = HU_2017.lon.apply(lambda x: -x)
data = pd.concat([data, HU_2017]).reset_index(drop=True) 

Assign zones according to measurement's latitude and longitude (see figure 1)

In [8]:
data['lat_lon'] = list(zip(data.lat, data.lon))
data.loc[:, 'A'] = data.lat_lon.apply(lambda x: 1 if (28.0 > x[0] >= 25.9) & 
                                      (-99.1 < x[1] <= -95.9) else 0)
data.loc[:, 'B'] = data.lat_lon.apply(lambda x: 1 if (30.9 > x[0] >= 28.0) & 
                                      (-97.6 < x[1] <= -92.4) else 0)
data.loc[:, 'C'] = data.lat_lon.apply(lambda x: 1 if (31.2 > x[0] >= 28.9) & 
                                      (-92.4 < x[1] <= -88.5) else 0)
data.loc[:, 'D'] = data.lat_lon.apply(lambda x: 1 if (31.4 > x[0] >= 28.9) & 
                                      (-88.5 < x[1] <= -85.1) else 0)
data.loc[:, 'E'] = data.lat_lon.apply(lambda x: 1 if (31.2 > x[0] >= 28.9) & 
                                      (-85.0 < x[1] <= -79.9) else 0)
data.loc[:, 'F'] = data.lat_lon.apply(lambda x: 1 if (28.9 > x[0] >= 26.7) & 
                                      (-83.3 < x[1] <= -80.0) else 0)
data.loc[:, 'G'] = data.lat_lon.apply(lambda x: 1 if (26.7 > x[0] >= 24.4) & 
                                      (-82.6 < x[1] <= -79.0) else 0)
data.loc[:, 'H'] = data.lat_lon.apply(lambda x: 1 if (34.0 > x[0] >= 31.2) & 
                                      (-82.4 < x[1] <= -78.3) else 0)
data.loc[:, 'I'] = data.lat_lon.apply(lambda x: 1 if (37.4 > x[0] >= 34.0) & 
                                      (-80.7 < x[1] <= -75.4) else 0)
data.loc[:, 'J'] = data.lat_lon.apply(lambda x: 1 if (41.6 > x[0] >= 37.4) & 
                                      (-78.8 < x[1] <= -72.0) else 0)
data.loc[:, 'K'] = data.lat_lon.apply(lambda x: 1 if (45.4 > x[0] >= 40.7) & 
                                      (-72.0 < x[1] <= -67.5) else 0)
data.drop(['lat_lon'], axis=1, inplace=True)

Any double-dipping between zones?

In [9]:
data[(data.A + data.B + data.C + data.D + data.E + data.F + data.G + data.H + data.I + \
      data.J + data.K) > 1].count().sum()

0

Save progress

In [10]:
data.to_csv('./data/hurdat2_cleaned.csv')

Create list of 'New Event' events. This will serve as a point for the beginning of each storm event chain.

In [11]:
num = []
events_index = np.array(data[data.status == 'NE'].index)
for i in range(1, 1832):
    num.append(events_index[i] - events_index[i - 1])
num += [51]    # Last storm event in dataset

Create list of prior measurements feature for each measurement in a storm event chain

In [12]:
num_meas = []
temp = num
for i in data.index:
    if data.status[i] == 'NE':
        count = num[0]
        sub_count = count
        del temp[0]
    else:
        sub_count -= 1
    num_meas.append(count - sub_count)
data.loc[:, 'prior_measures'] = num_meas

Each measurement is assigned a impact zone values based the storm's future, present, or past zone impacts

In [13]:
status_update = []
counter = 0
while counter < 51571:
    if data.status[counter] == 'NE':
        counter += 1
        stop = 0
        stat_ary = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        while data.status[counter] != 'NE' or stop > 150:
            stat_ary += np.array(data[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                                       'I', 'J', 'K']].iloc[counter])
            counter += 1
            stop += 1
    else:
        print('***************', list(data.iloc[counter]))
        counter += 1
    status_update.append([counter, np.sign(stat_ary)])

Updating dataframe with impact zone information for each measurement

In [29]:
status_mat = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
for _ in range(status_update[0][0] - 1):
    status_mat = np.vstack((status_mat, status_update[0][1]))
for i in range(1, len(status_update)):
    for _ in range(status_update[i][0] - status_update[i-1][0]):
        status_mat = np.vstack((status_mat, status_update[i][1]))
status_mat = np.vstack((status_mat, np.array(data[['A', 'B', 'C', 'D', 'E', 'F', 'G',
                                                   'H', 'I', 'J', 'K']].iloc[51572:])))
data[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']] = status_mat
data.iloc[51573:].A = data.iloc[51573:].A.apply(lambda x: 1)
data.iloc[51573:].B = data.iloc[51573:].B.apply(lambda x: 1)

/home/raven/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Adding previous 7 measurements for each measurement (lagged features)

In [34]:
data = pd.get_dummies(data, columns=['status'], prefix_sep='')
cols = ['lat', 'lon', 'max_wind', 'td', 'statusDB', 'statusEX', 'statusHU', 'statusLO',
        'statusNE', 'statusSD', 'statusSS', 'statusTD', 'statusTS', 'statusWV']
data = pd.concat([data,
                  data[cols].shift(-1),
                  data[cols].shift(-2),
                  data[cols].shift(-3),
                  data[cols].shift(-4),
                  data[cols].shift(-5),
                  data[cols].shift(-6),
                  data[cols].shift(-7)],
                 axis=1).iloc[:-7]    # Drop NaN rows

Renaming columns

In [35]:
data.columns = ['lat', 'lon', 'max_wind', 'year', 'month', 'day', 'hour',
       'td', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
       'I', 'J', 'K', 'prior_measures', 'statusDB', 'statusEX', 'statusHU',
       'statusLO', 'statusNE', 'statusSD', 'statusSS', 'statusTD', 'statusTS',
       'statusWV', 'lat1', 'lon1', 'max_wind1', 'td1', 'statusDB1',
       'statusEX1', 'statusHU1', 'statusLO1', 'statusNE1', 'statusSD1', 'statusSS1',
       'statusTD1', 'statusTS1', 'statusWV1', 'lat2', 'lon2', 'max_wind2',
       'td2', 'statusDB2', 'statusEX2', 'statusHU2', 'statusLO2', 'statusNE2',
       'statusSD2', 'statusSS2', 'statusTD2', 'statusTS2', 'statusWV2',
       'lat3', 'lon3', 'max_wind3', 'td3', 'statusDB3', 'statusEX3',
       'statusHU3', 'statusLO3', 'statusNE3', 'statusSD3', 'statusSS3', 'statusTD3',
       'statusTS3', 'statusWV3', 'lat4', 'lon4', 'max_wind4', 'td4',
       'statusDB4', 'statusEX4', 'statusHU4', 'statusLO4', 'statusNE4', 'statusSD4',
       'statusSS4', 'statusTD4', 'statusTS4', 'statusWV4', 'lat5', 'lon5',
       'max_wind5', 'td5', 'statusDB5', 'statusEX5', 'statusHU5', 'statusLO5',
       'statusNE5', 'statusSD5', 'statusSS5', 'statusTD5', 'statusTS5', 'statusWV5',
       'lat6', 'lon6', 'max_wind6', 'td6', 'statusDB6', 'statusEX6',
       'statusHU6', 'statusLO6', 'statusNE6', 'statusSD6', 'statusSS6', 'statusTD6',
       'statusTS6', 'statusWV6', 'lat7', 'lon7', 'max_wind7', 'td7',
       'statusDB7', 'statusEX7', 'statusHU7', 'statusLO7', 'statusNE7', 'statusSD7',
       'statusSS7', 'statusTD7', 'statusTS7', 'statusWV7']
data.drop(['statusNE', 'statusNE1', 'statusNE2', 'statusNE3', 'statusNE4', 'statusNE5',
           'statusNE6', 'statusNE7'], axis=1, inplace=True)

Filtering out 'New Event' rows (no longer needed)

In [36]:
data = data[data.month != 99]

One-hot encoding categorical features

In [37]:
data = pd.get_dummies(data, columns=['year', 'month', 'day', 'hour'], prefix_sep='')

Filtering out measurements that do not have at least seven lagged measurements

In [38]:
data = data[data.prior_measures >= 8]
data.drop(['prior_measures'], axis=1, inplace=True)

Dropping 'index' column (not needed)

In [39]:
data = data.reset_index().drop('index', axis=1)

Any missing values?

In [40]:
data.isnull().sum().sum()

0

Saving data frame

In [41]:
data.to_csv('./data/hurdat2_preprocessed.csv')